Nombre 1: García Sánchez, Sergio

Nombre 2: Ruiz Nieto, Miguel Emilio

# Sistemas de Gestión de Datos y de la Información 

## Práctica 1

- Se valorará la claridad del código y evitar redundancias o código poco eficiente
- Además de las funciones que se piden se pueden añadir otras auxiliares

**Ejercicio 1** [5 puntos]

a) [1] Escribir una función `cargadatos` que cargue en un dataframe de pandas  el contenido de un fichero csv dado con una codificación dada.

- *Nombre*: `cargadatos`
- *Parámetros de entrada*: 

    - `fichero`: path al fichero csv (puede ser un path local o una url)
    -  `codificacion`: codificación de caracteres del fichero, por ejemplo 'utf-8'. Ver [aquí](https://docs.python.org/3/library/codecs.html#standard-encodings) la lista de posible codificaciones
    
- *Devuelve*: el dataframe con los datos 

En el ejemplo se carga un dataframe con datos de usuarios de Twitter incluyendo sus coordenadas

In [2]:
import pandas as pd


# solucion
def cargadatos(fichero,codificacion):
    return pd.read_csv(fichero, encoding = codificacion)

# no borrar esto para ver el resultado
fichero =  "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/user_coords.csv"
codificacion = "utf-8"
df = cargadatos(fichero, codificacion)
df

,id,screen_name,location,n_tweets,followers,friends,long,lat
0,723592172721836032,sagidario81,La Orotava,6,9,197,-16.595364,28.260727
1,2834102043,lluviaEnVlc,València,26,335,5,-0.375000,39.466667
2,138264123,Jalex5,Madrid,1,243,364,-3.683330,40.400000
3,1354010144850530305,spainmunic,Valle de las Navas,160,52,6,-3.583367,42.472236
4,106909650,marcelogongora,Sevilla,1,567,324,-5.983333,37.383333
...,...,...,...,...,...,...,...,...
1624,516924028,ro2studio,Plasencia,1,75,79,-6.090080,40.032270
1625,191241567,Ivan_Parra_7,Villarino de los Aires,1,339,1002,-6.464460,41.268820
1626,1317983919703314432,DressageTH,Pamplona/Iruña,1,16,114,-1.648400,42.800290
1627,91317326,djabelramos,Madrid,1,99316,174,-3.683330,40.400000


b)[1] EScribir una función `iniciapagina` que arranque chromedriver y cargue una página dada. El parámetro `chromedriver` indica el path al fichero `chromedriver.exe`

- *Nombre*: `iniciapagina`
- *Parámetros de entrada*: 
    - `chromedriver`: el path al fichero chromedriver.exe
    - `pagina`: un string con la url de la página a cargar
    
- *Devuelve*: el driver tras cargar la página

En el ejemplo se abre una página que permite calcular la distancia geográfica entre dos puntos

In [5]:
# Ahora arrancamos una instancia de google Chrome. 
import os
from selenium import webdriver  # si da error, desde anaconda prompt sobre python 3.x hacer pip install --user  selenium
# solución
def iniciapagina(chromedriver,pagina):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--no-sandbox')
  driver = webdriver.Chrome(executable_path=chromedriver,options=chrome_options)
  driver.get(pagina)
  return driver

# para probar....

# chromedriver = "c:/hlocal/sgdi/chromedriver.exe"
# cambiar por la dirección en la que tengamos chromedriver.exe
# chromedriver = Constants.CHROME_DRIVER_PATH
# chromedriver = "/home/miguel/.local/bin/chromedriver"
chromedriver = "/Users/tronxi/workspace/sgdi/chromedriver"
print(chromedriver)
pagina = "https://www.nhc.noaa.gov/gccalc.shtml"
driver = iniciapagina(chromedriver,pagina)
# driver.close()

/Users/tronxi/workspace/sgdi/chromedriver


c) [3p] Escribir una función `distancia(driver,lat1,lon1,lat2,lon2)` que dadas las coordenadas de dos puntos (lat1,long1) y (lat2,long2) nos devuelva la distancia en km que las separa, utilizando para el cálculo la página que hemos abierto y cuyo acceso está en driver.

- *Nombre*: distancia(driver,lat1,long1,lat2,long2)
- *Parámetros de entrada*: 

    - driver: el driver selenium que permite acceder a la página que hemos abierto
    - lat1: la latitud del primer punto, de -90 a +90
    - lon1: la longitud del primero punto, de -180 a +180
    - lat2: la latitud del segundo punto, de -90 a +90
    - lon2: la longitud del segundo punto, de -180 a +180
    
- *Devuelve*: la distancia en km como un entero

**Nota**: Hay que tener en cuenta que la forma de indicar las coordenadas en la latitud y longitud varía, por lo que tendremos que examinar, y en su caso adaptar, los parámetros de entrada. En particular:

- Aunque la latitud de entrada va de -90 a 90, en la página siempre hay que poner un valor de 0 a 90, indicando si es latitud norte (N) o sur (S) en la lista desplegable. Por ejemplo, el parámetro -45 se convertira en +45 eligiendo del desplegable S, mientras el parámetro +45 se quedará tal cual pero eligiendo de la lista N.

- Análogamente, mientras que la latitud va de -180 a +180, en la página los números negativos se convertirán en positivos seleccionando W de su lista desplegable, mientras que los positivos no se modifican pero habrá que elegir E de su lista desplegable.

In [94]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

def calcular_hemisferio(lat):
  if lat >= 0:
    return 'N'

  return 'S'

def calcular_angulo(long):
  if long >= 0:
    return'E'

  return 'W'

def to_str_in_abs(number):
  return str(abs(number))

def replace_value_by_name(driver, name, value):
  element = driver.find_element_by_name(name)
  element.clear()
  element.send_keys(value)

def set_visible_value_in_select_by_name(driver, name, value):
    Select(driver.find_element_by_name(name)).select_by_visible_text(value)

def distancia(driver,lat1,lon1,lat2,lon2):
  hemisferio1 = calcular_hemisferio(lat1)
  lat1 = to_str_in_abs(lat1)
  angulo1 = calcular_angulo(lon1)
  lon1 = to_str_in_abs(lon1)

  hemisferio2 = calcular_hemisferio(lat2)
  lat2 = to_str_in_abs(lat2)
  angulo2 = calcular_angulo(lon2)
  lon2 = to_str_in_abs(lon2)

  replace_value_by_name(driver, "lat1", lat1)
  replace_value_by_name(driver, "lon1", lon1)
  set_visible_value_in_select_by_name(driver, "NS1", hemisferio1)
  set_visible_value_in_select_by_name(driver, "EW1", angulo1)

  replace_value_by_name(driver, "lat2", lat2)
  replace_value_by_name(driver, "lon2", lon2)
  set_visible_value_in_select_by_name(driver, "NS2", hemisferio2)
  set_visible_value_in_select_by_name(driver, "EW2", angulo2)
  
  set_visible_value_in_select_by_name(driver, "Dunit", "km")

  driver.find_element_by_xpath("//form[@name='OutputFormCD']/center/p/input[@value='Compute']").click()
  return driver.find_element_by_name("d12").get_attribute("value")

In [95]:
# para probar
driver = iniciapagina(chromedriver, pagina)
print(distancia(driver,90,0,-90,0)) # 20002 aprox
print(distancia(driver,-90,0,90,0)) # 20002 aprox
print(distancia(driver,0,-180,0,+180)) # 0 aprox
print(distancia(driver,0,180,0.001,0)) # 20001 aprox
print(distancia(driver,40.4167, -3.70325 ,41.3879,2.16992)) # 505
driver.close()

20002
20002
0
20001
505


**Ejercicio 2** (5 puntos)

a) [1] Escribir una función `datos_estaciones(api_key)` que nos permita obtener los datos de todas las estaciones metereológicas de la red española (se adjunta la función `datos_meteo` para que se use) y la forma de obtenerla se encuentra en el notebook api-meteo, basta con copiar la información y darle forma de función)

- *Nombre*: datos_estaciones(api_key)
- *Parámetros de entrada*: 

    - api_key: nuestra clave para accerder a la API metereológica, si no la tienes puedes conseguirla [aquí](https://opendata.aemet.es/centrodedescargas/altaUsuario?)
    
    
- *Devuelve*: los datos de las estaciones. Ojo: solo los datos, no los metadatos

In [ ]:
import json # para convertir string a JSON
import requests # para descargar ficheros
import pprint # para mostrar JSON en "bonito"


# datos metereológicos, recibe la url para el servicio concreto y la clave de api
def datos_meteo(url,api_key):
    querystring = {"api_key":api_key}
    response = requests.get( url,  params=querystring)
    #print(response.headers)
    respuesta = json.loads(response.text)
    #print("Paso 1")
    print("paso 1 ",respuesta)

    if respuesta["estado"]==200: # éxito
        #print("Paso 2")
        pet_datos = requests.get(respuesta['datos'])
        #print("respuesta a la peticion de datos",pet_datos,pet_datos.text)
        valores_datos = json.loads(pet_datos.text)
        #print("Datos: ")
        #pprint.pprint(valores_datos)
        pet_metadatos = requests.get(respuesta['metadatos'])
        valores_metadatos = dict(json.loads(pet_metadatos.text))
        #print("="*50)
        #print("Metadatos: ")
        #pprint.pprint(valores_metadatos)
    else:
        valores_datos = None
        valores_metadatos = None
    return valores_datos,valores_metadatos


### Solución
def datos_estaciones(api_key):
    

#### para probar poned vuestra clave...
api_key = ""
estaciones = datos_estaciones(api_key)
len(estaciones),estaciones[:3]

b) [2] La variable `estaciones` es una lista de diccionarios Python, cada uno con claves

- 'altitud': altitud de la estación
- 'indicativo': identificador único de la estación
- 'indsinop': Indicativo sinóptico
- 'latitud': latitud de la estación
- 'longitud': longitud de la estación
- 'nombre': nombre que recibe la estación
- 'provincia': provincia en la que se encuentra

Queremos escribir una función `estacion_cercana(driver,df,screen_name,estaciones)` que dado un usuario de los del dataframe df, extraiga sus coordenadas y encuentre la estación metereológica más cercana de entre las que se le pasen como parámetro.

- *Nombre*: estacion_cercana(driver,df,screen_name,estaciones)
- *Parámetros de entrada*: 

    - driver: acceso a la página para calcular distancias a partir de coordenadas
    - df: dataframe de usuarios que cargamos en el ejercicio 1
    - screen_name un valor de esta columna en df, es decir el nombre de un usuario de df en Twitter. Asumimos que existe en df.
    - estaciones: la lista de estaciones metereológicas obtenida en el ejercicio anterior
    
    
- *Devuelve*: El `indicativo` de la estación más cercana, las coordenadas de esta estación, las coordenadas del usuario y la distancia que separa al usuario de esta estación 

Notas: 
- Utilizar las funciones que hemos hecho hasta ahora, no repetir código
- En las estaciones la longitud y la latitud son strings con un formato diferente al que requiere la función `distancia`. Para convertir cada valor en el formato que se requiere en esta función puedes utilizar la función `convierte`

In [ ]:
def convierte(tud):
    factor = 1 if tud[-1] in ['N', 'E'] else -1
    return factor * sum(float(x) / 60 ** n for n, x in enumerate([tud[0:2],tud[2:4],tud[4:6]]))

print(convierte('412316N'))
print(convierte('021012E'))

In [ ]:
# solución
def estacion_cercana(driver,df,screen_name,estaciones):

    
    
# para probar
screen_name = 'ro2studio'
# nos quedamos con algunas estaciones, para que tarde menos
algunas = [b for a,b in enumerate(estaciones) if a %15==0]
indMax,distMax,lat1,lon1 = estacion_cercana(driver,df,screen_name,algunas)   
indMax,distMax,lat1,lon1

c) [1] Utilizar la librería `folium` para definir una función `estaciones_y_usuario(estaciones,screen_name,lat1,lon1)` que dado un conjunto de estaciones, un nombre de usuario y las coordenadas del usuario nos devuelva un mapa donde vengan representados tanto las estaciones como el usuario. La leyenda asociado a las estaciones tiene que ser su nombre + su indicativo. En el caso del usuario, que debería aparecer con un icono diferente, sería el nombre del usuario.  

- *Nombre*: `estaciones_y_usuario(estaciones,screen_name,lat1,lon1)` 
- *Parámetros de entrada*: 

    - estaciones: conjunto de estaciones metereológicas
    - screen_name: el nombre de un usuario
    - lat1, lon1: las coordenadas de este usuario
    
- *Devuelve*: Un map de folium con la representación de las estaciones y del usuario como se indica más arrib. Para crear el mapa partir de la `location` [36,-6] con un `zoom_start` de 5.

Notas: es fácil encontrar información de folium en la web. Por ejemplo se puede utilizar:

https://python-visualization.github.io/folium/quickstart.html

In [ ]:
import folium

# solución
def estaciones_y_usuario(estaciones,screen_name,lat1,lon1):
    

In [ ]:
# para probar
estaciones_y_usuario(algunas,screen_name,lat1,lon1)

d) [1] Queremos hacer un gráfico sencillo que incluya las temperaturas diarias máxima y mínima para la estación 2117D', desde el 01 de enero 2019 hasta el 31 de diciembre del mismo año.

- *Nombre*: `gráfico_de_temperaturas(clave_api,estacion,fecha_ini,fecha_fin)` 
- *Parámetros de entrada*: 

    - clave_api: clave secreta para acceder a los datos metereológicos
    - estacion: el indicativo de la estación
    - fecha_ini,fecha_fin: dos strings que indican la fecha inicial y la fecha final, Ambas con el formato 'yyyy-mm-dd'
    
- *Devuelve*: Nada, simplemente muestra las gráficas

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# solución
def gráfico_de_temperaturas(clave_api,estacion,fecha_ini,fecha_fin):


In [ ]:
# para probar
estacion='2117D'
fecha_ini = '2019-01-01'
fecha_fin = '2019-12-31'
gráfico_de_temperaturas(api_key,estacion,fecha_ini,fecha_fin)